In [33]:
import numpy as np
import pandas as pd
import math as m
import cmath
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint
from scipy.optimize import minimize as GDlib
import MLE_functions as fn
from scipy.optimize import fsolve
from scipy.optimize import least_squares

In [34]:
pr_num = [sym.lambdify((fn.theta1, fn.theta2, fn.phi1, fn.phi2, fn.alpha), i) for i in fn.pr]   # lambdify the probabilities

# checking probabilities syum to 1
# creating fucntion to check at 100 random points the sum of the probs to see if any of them dont sum to 1
def check_sum(xx):
    sum_pr_list = []
    params_list = []
    for i in range(xx):
        theta1 = rand.uniform(0, m.pi)
        theta2 = rand.uniform(0, m.pi)
        phi1 = rand.uniform(0, m.pi)
        phi2 = rand.uniform(0, m.pi)
        alpha = rand.uniform(0, m.pi)
        params = [theta1, theta2, phi1, phi2, alpha]
        sum_pr = sum([i(theta1, theta2, phi1, phi2, alpha) for i in pr_num])
        
        params_list.append(params)
        sum_pr_list.append(sum_pr)
        
        #zip and sort in the order of sum of the probabilities
        zipped_lists = zip(sum_pr_list, params_list)
        sorted_zipped_lists = sorted(zipped_lists)
        
    return sorted_zipped_lists

sorted_zipped_lists = check_sum(50)

print(sorted_zipped_lists)

[(1.0000000000000013, [2.265296877986555, 1.8918211384912382, 1.3916056580056657, 0.5945610256119627, 0.6667857953922188]), (1.0000000000000013, [2.3177043743113135, 1.0861105600070926, 1.4171994435824677, 2.7598348605383687, 1.4717224455442677]), (1.0000000000000016, [1.0533169989245574, 2.3794407509967326, 2.2734063354534615, 1.6308158858652484, 2.0948104537749885]), (1.0000000000000016, [2.6692081392213094, 0.43303405565779046, 0.2544337544362274, 2.538710432846454, 0.9381217268962634]), (1.0000000000000018, [0.008143607094407947, 0.8395944110464606, 1.2494890890600456, 1.224722634865263, 0.2181002179750774]), (1.0000000000000018, [0.19710741559889922, 2.8758734268985156, 2.3147466538819654, 1.090173809309891, 2.4741186138400684]), (1.0000000000000018, [0.36892030171070755, 1.9823010590846581, 2.547536588552738, 1.0245634347111645, 2.307305078649268]), (1.0000000000000018, [0.7673745811328441, 0.8319143622836367, 0.7751357484730328, 1.8200063217718792, 1.4797335759610222]), (1.00000

In [3]:
# Improving the previous loss function, new code: 
def loss_new(vars, params_tru = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, nj_unnorm = None, lossfn = 'new'):                         
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_tru, N = N)                             # for getting the collapse count
        n_unnorm = num_expt[4]

    n = [i/sum(n_unnorm) for i in n_unnorm]                                                # normalise the counts
    eq_pr = [pr_num[i](*vars) for i in range(len(pr_num))]
    
    lnl_p = [i * np.log(p) if i != 0 else 0 for i, p in zip(n, eq_pr)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.
    lnl_n = [i * np.log(i) if i != 0 else 0 for i in n]                         
    del_lnl = [(k-l) for k,l in zip(lnl_p, lnl_n)]
    loss = sum([abs(i) for i in del_lnl])

    if lossfn == 'old':
        loss = sum([abs(i) for i in lnl_p])

    return loss


In [4]:
# the gradient function
def grad(vars, params_i = None ,nj_unnorm = None, s=(1/500), N = 10000, loss = 'new'):    # "point" is the list of parameters
    if nj_unnorm != None:
        n_unnorm = nj_unnorm
    else:
        num_expt = fn.num_experiment(params= params_i, N = N)              # for getting the collapse count
        n_unnorm = num_expt[4]
    
    del_theta1 = loss_new(vars = [vars[0]+s,vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_theta2 = loss_new(vars = [vars[0],vars[1]+s,vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_phi1   = loss_new(vars = [vars[0],vars[1],vars[2]+s,vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_phi2   = loss_new(vars = [vars[0],vars[1],vars[2],vars[3]+s,vars[4]], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    del_alpha  = loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]+s], nj_unnorm= nj_unnorm, lossfn = loss) - loss_new(vars = [vars[0],vars[1],vars[2],vars[3],vars[4]], nj_unnorm= nj_unnorm, lossfn = loss)
    
    grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
    grad = [i if abs(i) > 1e-12 else 0 for i in grad]
    
    xx = 5
    return grad


In [5]:
# original working GD function
def GD_new(params_i, params_true = [0, m.pi/2, 0, 0, m.pi/4], nj_unnorm = None, nit = 20, step = 1/120, N = 10000, loss = 'new'):
    if nj_unnorm == None:                                                           # priority is using the given collapses, else build using fn.num_expt
        nj_unnorm = fn.num_experiment(params= params_true, N = N, an_pr= True)[4]           
    params = params_i
    loss_i = loss_new(vars = params, nj_unnorm= nj_unnorm)                                          # Initial parameters, loss
    
    params_history = [params]                                          # To store the history of parameters, loss and steps
    loss_history = [loss_i]                                         
    steps = [step]
    for i in range(nit):
        if loss == 'new':
            gradients = grad(vars = params, nj_unnorm= nj_unnorm)
        else:
            gradients = grad(vars = params, nj_unnorm= nj_unnorm, loss = 'old')
        params = [(i - step*grad) for i, grad in zip(params, gradients)]    # Basically, same as entry by entry updating the parameters.
        
        if i == (.5*nit)//1:                                   # variable step size after 50% and 75% of the iterations
            step = step/2
        if i == (.75*nit)//1:
            step = step/5
        steps.append(step)
        params_history.append(params)
        loss_history.append(loss_new(vars = params, nj_unnorm= nj_unnorm))
        if sum(abs(m-n) for m,n in zip(params, params_history[-2])) < 1e-10:
            print(f'Gradient descent converged at {params} after {i} iterations')
    xx = 6                                                          # dummy for debugging
    return params


In [6]:
#adding a randomized function for pure gradient descent without using the inversion. The idea is that this function will take true parameters, number of iterations, number of random trials, etc. as arguments and it will basically choose randomly that many number of lists and then run the previously defined gradient descent function gd underscore new on each of those and add the resultant parameters and the corresponding fidelities to lists and then it will pick out the one with the best fidelity and the corresponding parameters and return them
def gd_rand(params_tru = [0, m.pi/2, 0, 0, m.pi/4], N = 10000, nj_unnorm = None, loss = 'new', step = 1/3, nit = 100, rat = 30):
    params_list = []
    fid_list = []
    for i in range(rat):
        params_i = [rand.uniform(0,m.pi) for i in range(5)]
        opt_params_i = GD_new(params_i = params_i, params_true = params_tru, N = N, nj_unnorm = nj_unnorm, loss = loss, nit = nit, step = step)
        fid_list.append(fn.fid(opt_params_i, params_tru))
        params_list.append(opt_params_i)

    # zip the lists
    results_list = zip(fid_list, params_list)
    
    # sort and pick the best ten percent results
    results_list = sorted(results_list, key = lambda x: x[0], reverse = True)
    best_results = results_list[:rat//10]                       # best 10% of the fidelities and the corresponding parameters
    
    best_params = [i[1] for i in best_results]
    best_fids = [i[0] for i in best_results]

    return (best_params, best_fids)


In [7]:
# feed it the true parameters, it will give results comparing the parameters and the fidelities of the inversion method and the hybrid method
def results(params_tru, nit = 100, step = 1/70, N = 10000):
    inv = fn.Inversion_new(params = params_tru, N = N, threshold = 'variable')
    nj_unnorm = inv[6]
    opt_inv = inv[2]
    # inv_fid_c = inv[0]                              # using coeffcients from the inversion protocol to calculate fidelity
    fid_inv = fn.fid(opt_inv, params_tru)        # using parameters to calculate fidelity, should be same as from the coefficients
    
    opt_GD = GD_new( params_i = opt_inv , nj_unnorm = nj_unnorm, nit = nit, step = step, N = N)
    fid_opt = fn.fid(opt_GD, params_tru)
    
    ppm_inv_fid = [(1- i)*1e6 for i in fid_inv]     # parts per million error in fids
    ppm_opt_fid = [(1- i)*1e6 for i in fid_opt]
    
    # print(f'tru_params_deg: {[i*(180/m.pi) for i in params_tru]} \n tru_params: {params_tru} \n opt_inv: {opt_inv} \n opt_GD: {opt_GD} \n \n  inv_fid: {fid_inv} \n GD_fid: {fid_opt}  \n\n ppm_inv_fid: {ppm_inv_fid} \n ppm_opt_fid: {ppm_opt_fid} \n')

    return [params_tru, nj_unnorm, opt_inv, opt_GD , fid_inv, fid_opt, ppm_inv_fid, ppm_opt_fid]   


In [40]:
# trying the old loss, now wwith the correct probabilities

# params_i = [rand.uniform(0,m.pi) for i in range(5)]
params_i = fn.Inversion_new(params = [m.pi/4, m.pi/2+m.pi/4, 0, 0, m.pi/4], N = 10000, threshold = 'variable')[2]
params_tru = [m.pi/4, m.pi/2+m.pi/4, 0, 0, m.pi/4]
# opt_p = GD_new(params_i = params_i, params_true = params_tru, N = 10000, loss = 'old', nit = 50, step = 1/5)
opt_p = GD_new(params_i = params_i, N = 10000, nit = 50, step = 1/5)

params_i_degrees = [i*(180/m.pi) for i in params_i] 
opt_p_degrees = [i*(180/m.pi) for i in opt_p]

print('\n tru params ', [i*(180/m.pi) for i in params_tru] ,'\n opt_p_degrees:', opt_p_degrees, '\n params_i_degrees:', params_i_degrees)

fids_opt = fn.fid(opt_p, params_tru)
fids_inv = fn.fid(params_i, params_tru)
fids_opt, fids_inv


 tru params  [45.0, 135.0, 0.0, 0.0, 45.0] 
 opt_p_degrees: [360.0178561892273, 89.71741720969744, 124.52625583597869, -154.54877062635077, (44.59358455611585-7.67504195354202e-17j)] 
 params_i_degrees: [45.413971053989975, 44.97208496779245, 0.5324424926602901, -179.92245596910786, (44.65140134482599-7.67504195354202e-17j)]


([0.49982336222981805, 0.5044532842364494],
 [0.9999262110315976, 0.9999993047054998])

In [ ]:
# Trying another way simplifying the laws at many many many random points and picking the ones in which the loss is really close to the theoretical loss and then refining by making steps smaller around each of those points until we reach the threshold precision let's say 1e-5 in the loss function.


In [35]:
#try
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
results(params_tru1, nit = 300, step = 1/200, N = 10000)

tru_params_deg: [29.999999999999996, 119.99999999999999, 45.0, 90.0, 29.999999999999996] 
 tru_params: [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988] 
 opt_inv: [0.5257531629285364, 1.012673773364041, 0.7943980307830147, -1.5594632130249584, (0.5353590075198238+2.1985963773791503e-17j)] 
 opt_GD: [0.5278778329628031, 1.0283658526741655, 0.8039305470572345, -1.5573297830992687, (0.525432692914733+2.1985963773791503e-17j)] 
 
  inv_fid: [0.9999801340334321, 0.9987835982022493] 
 GD_fid: [0.9999169790745976, 0.9996106917634611]  

 ppm_inv_fid: [19.86596656788908, 1216.4017977507324] 
 ppm_opt_fid: [83.02092540235506, 389.3082365389322] 



In [17]:
# retrying, above is the oringinal one
#try
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
res = results(params_tru1, nit = 300, step = 1/200, N = 10000)

print('tru = ', res[0])

print('nj_unnorm = ', res[1])
print('opt_inv = ', res[2])
print('opt_GD = ', res[3])
print('fid_inv = ', res[4])
print('fid_GD = ', res[5])
print('ppm_inv = ', res[6])
print('ppm_GD = ', res[7])


"""
# one saved result for params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
tru =  [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988]  # same as [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]
nj_unnorm =  [654, 558, 1169, 967, 2313, 521, 1260, 1152, 1406]
opt_inv =  [0.5254370610538157, 1.030984356143689, 0.7670544953445808, -1.6068690955721805, (0.5346630644404312+2.5302074683784063e-17j)]
opt_GD =  [0.5320294066145164, 1.0327066070674442, 0.7689496798867287, -1.6064174644839841, (0.5366920641304006+2.5302074683784063e-17j)]
fid_inv =  [0.9999333970821788, 0.9994887602573697]
fid_GD =  [0.9998777120836085, 0.9995482600825005]
ppm_inv =  [66.60291782123463, 511.2397426303428]
ppm_GD =  [122.28791639146141, 451.73991749947186] 
"""

rtu =  [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988]
nj_unnorm =  [654, 558, 1169, 967, 2313, 521, 1260, 1152, 1406]
opt_inv =  [0.5254370610538157, 1.030984356143689, 0.7670544953445808, -1.6068690955721805, (0.5346630644404312+2.5302074683784063e-17j)]
opt_GD =  [0.5320294066145164, 1.0327066070674442, 0.7689496798867287, -1.6064174644839841, (0.5366920641304006+2.5302074683784063e-17j)]
fid_inv =  [0.9999333970821788, 0.9994887602573697]
fid_GD =  [0.9998777120836085, 0.9995482600825005]
ppm_inv =  [66.60291782123463, 511.2397426303428]
ppm_GD =  [122.28791639146141, 451.73991749947186]


In [23]:
params_tru1 = [m.pi/6, m.pi/2+m.pi/6, m.pi/4, m.pi/2, m.pi/6]   # degrees : 30, 120, 45, 90, 30
gd_rand(params_tru = params_tru1, N = 10000, nit = 200, rat = 50, step = 1/4)

([[2.620468715033219,
   2.096494845467305,
   3.9114857464288484,
   1.4589030314243545,
   0.5408658293314748],
  [0.5307330358786534,
   2.0624357350425906,
   0.7728786246881507,
   1.5799142276391531,
   0.5169653275031013],
  [2.610313065974753,
   2.0650790496323403,
   3.9417660898299878,
   1.6401453721317285,
   0.510908991325928],
  [3.6531942178637054,
   2.0773711313590955,
   0.7944103700028262,
   1.5921864340418774,
   2.6174525639468436],
  [0.5128119471599131,
   2.0927995741456185,
   0.7593622889818816,
   1.5724015950617436,
   2.596075597410029]],
 [[0.9999489291082766, 0.9976448536837781],
  [0.9999194759454406, 0.998963965320031],
  [0.9998997159677367, 0.9982714789693796],
  [0.9998410605320175, 0.999626163265139],
  [0.9997581684718029, 0.9999969720183342]])

In [75]:
martin_p =[ 0.5264971926438728, 2.621069253994012, 2.634918539482526 ,2.595433045991269, 0.760913797157194 ]
p_tru= [0, m.pi/2, 0, 0, m.pi/4]

fn.fid(p_tru, martin_p)

[0.7474857108667748, 0.24734139200269095]

In [8]:
#rough
njn = [805, 1593, 836, 803, 1705, 851, 841, 1732, 834]
[0,1.570796326794897, 0,0, 0.7853981633974483]

ll = loss_new(vars = [0,1.570796326794897, 0,0, 0.7853981633974483], nj_unnorm = [805, 1593, 836, 803, 1705, 851, 841, 1732, 834], lossfn = 'old')
ll
kk = loss_new(vars = [0.5264971896014186 ,0.5205234056820061, 2.595433049161852 ,5.776511197655469, 2.331710128012299 ], nj_unnorm = [805, 1593, 836, 803, 1705, 851, 841, 1732, 834], lossfn = 'old')
kk

2.164373601654298

In [9]:
#rough
prd = [round(i(0,1.570796326794897, 0,0, 0.7853981633974483),4) for i in pr_num]
# print(sum(prd))
print('prd: \n', prd)
n = [i/sum(njn) for i in njn]
print('n: \n', n)
# th min loss
loss_th_min = -sum([i*np.log(i) if i != 0 else 0 for i in n])

#old loss
lnl_p_list = [p * np.log(i) if i != 0 else 0 for i, p in zip(n, prd)]
lnl_p = sum([abs(i) for i in lnl_p_list])
print('loss at params list:\n ', lnl_p_list)
print('loss at params:\n ', lnl_p)


prd: 
 [0.0833, 0.0833, 0.1667, 0.0833, 0.0833, 0.1667, 0.0833, 0.0833, 0.1667]
n: 
 [0.0805, 0.1593, 0.0836, 0.0803, 0.1705, 0.0851, 0.0841, 0.1732, 0.0834]
loss at params list:
  [-0.2098741912766497, -0.15301927297010898, -0.41370135020720983, -0.21008140531385078, -0.1473593645221385, -0.4107368381752482, -0.20622986770986945, -0.14605057996172172, -0.4141006318352265]
loss at params:
  2.311153501972024


In [10]:
spec_params = [0,1.570796326794897, 0,0, 0.7853981633974483]
njn = [805, 1593, 836, 803, 1705, 851, 841, 1732, 834]
nn = [i/sum(njn) for i in njn]
spec_loss = loss_new(vars = spec_params, nj_unnorm = njn, lossfn = 'old')
print('spec_loss: ', spec_loss)

#prob comparison at spec params
prd = [round(i(*spec_params),4) for i in pr_num]
print('prd: \n', prd)
print('collapse prob: \n', nn)


spec_loss:  2.310164245568836
prd: 
 [0.0833, 0.0833, 0.1667, 0.0833, 0.0833, 0.1667, 0.0833, 0.0833, 0.1667]
collapse prob: 
 [0.0805, 0.1593, 0.0836, 0.0803, 0.1705, 0.0851, 0.0841, 0.1732, 0.0834]


In [11]:
# again comparing the probabilities but this time using abstract ones generated directly from computer not using professor hillary's ones.

rho_abs = fn.Creating_states(Abstract = 1)[2]        #abstract rho

#abstract probs
pr_abs = [np.trace(np.dot(fn.POVM_elts[i],rho_abs)) for i in range(9)]
# pr_abs[1]
pr_spec_abs = [i.subs({fn.theta1: spec_params[0], fn.theta2: spec_params[1], fn.phi1: spec_params[2], fn.phi2: spec_params[3], fn.alpha: spec_params[4]}) for i in pr_abs]

print('pr_spec_abstract: \n', pr_spec_abs)

# Since the abstract probabilities are the same as Professor Hillary's probabilities but different from the collapsed probabilities, the error must be somewhere in the collapse.

pr_spec_abstract: 
 [0.0833333333333334, 0.0833333333333333, 0.166666666666667, 0.0833333333333332 + 1.7563329547635e-18*I, 0.0833333333333333 - 1.14422459262848e-49*I, 0.166666666666667 + 1.7563329547635e-18*I, 0.0833333333333332 - 3.90308845140291e-19*I, 0.0833333333333333 + 5.14884401085484e-49*I, 0.166666666666667 - 3.90308845140289e-19*I]


In [12]:
# checking if the error is in the num_experiment function

expt = fn.num_experiment(params= [0,m.pi/2, 0,0, m.pi/4], N = 10000, an_pr= True)

th_probs= expt[3]
collapses = expt[4]
collapse_prob = [i/sum(collapses) for i in collapses]

print('th_probs: \n', th_probs)
print('collapses: \n', collapses)

# the collapses are correct, the error is in camillas probs..Now starting over using a fresh example to trace where and why does the loss exceed the minimum value.

th_probs: 
 [0.083333333333, 0.083333333333, 0.166666666667, 0.083333333333, 0.083333333333, 0.166666666667, 0.083333333333, 0.083333333333, 0.166666666667]
collapses: 
 [835, 833, 1625, 859, 881, 1650, 792, 825, 1700]


In [13]:
#writing a loop to filter out the highest loss iteratively
params_tru = [0, m.pi/2, 0, 0, m.pi/4]
n_fresh =  [842, 854, 1615, 799, 800, 1714, 892, 813, 1671]
n_fresh_norm = [i/sum(n_fresh) for i in n_fresh]
th_loss_max = -sum([i*np.log(i) if i != 0 else 0 for i in n_fresh_norm])
params_list = []
loss_list = []
for i in range(100):
    params_i = [rand.uniform(0,m.pi) for j in range(5)]
    loss = loss_new(vars = params_i, nj_unnorm = n_fresh, lossfn = 'old')
    params_list.append(params_i)
    loss_list.append(loss)

# zip the lists, pick max
results_list = zip(loss_list, params_list)
results_list = sorted(results_list, key = lambda x: x[0], reverse = True)
print('th max loss: ', th_loss_max)
print(results_list[:10])
# from here we see that at parameters [0.7067015919596139, 0.876436876817383, 1.6730930859371373, 0.26594776402832554, 0.30328983213952054] ,the loss is 2.3218 instead of the minimum possible 2.1377

th max loss:  2.1377889507324106
[(3.058975036141985, [0.07583647691419862, 3.0845492930528158, 2.554248964262986, 1.5478842657524106, 1.836657100525946]), (2.954898917583247, [1.5487356722524563, 1.4829360691778117, 0.8778573743716248, 1.3783192693168582, 2.2772220070995925]), (2.9057594156177355, [2.321815929385002, 3.0911693488166883, 0.1090113937111211, 1.145769653231649, 1.6769127917084168]), (2.878629547528605, [2.3052475650573574, 0.028609504657239427, 1.0236013879089865, 0.2347942431403365, 1.735105019665889]), (2.7704404843407007, [0.07923659679156346, 1.4696350150740096, 1.4315467330035465, 1.3287969705329492, 1.5832193353442658]), (2.6970623297891327, [0.7629700074232287, 0.8181669811855173, 0.342799465870813, 0.34022433283097975, 2.620180199046174]), (2.688638330630604, [2.4800239245082847, 0.08538550503363386, 1.7170642751022025, 0.7128891978922417, 1.7449483144863052]), (2.6883576996197482, [0.11701345146873242, 3.0059184508813828, 2.297166243252706, 2.8688048637349963, 2

In [14]:
# Now, finding where or how the loss is increasing. Probably there's a mismatch somehow in the collapsed probabilities and the theoretical probabilities. 
# Checking for that now at the special parameters found above.

spec_params = [0.7067015919596139, 0.876436876817383, 1.6730930859371373, 0.26594776402832554, 0.30328983213952054]
spec_params_nicer = [0, 1, 0, 0, m.pi/2]
loss_nice = loss_new(vars = spec_params_nicer, nj_unnorm = n_fresh, lossfn = 'old')
print('loss_nice: ', loss_nice)
# n_fresh =  [842, 854, 1615, 799, 800, 1714, 892, 813, 1671]

pr_an = [round(i(*spec_params_nicer),14) for i in pr_num]
print('pr_an: \n', pr_an)
print('collapse probs: \n', n_fresh_norm)

pr_spec_abs_sym = [i.subs({fn.theta1: spec_params[0], fn.theta2: spec_params[1], fn.phi1: spec_params[2], fn.phi2: spec_params[3], fn.alpha: spec_params[4]}) for i in pr_abs]
pr_spec_abs = [round(i.evalf(),10) for i in pr_spec_abs_sym]
print('pr_spec_abstract: \n', pr_spec_abs)
#It is crazy that the probabilities are coming out to be complex using the abstract method Something must be wrong in Basis arrangement...

print(sum(pr_an))

loss_nice:  2.7996275431571207
pr_an: 
 [0.00070640100534, 0.02053862432267, 0.0288630315947, 0.22834151721771, 0.11438869642226, 0.13221575789868, 0.22834151721771, 0.11438869642226, 0.13221575789868]
collapse probs: 
 [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]
pr_spec_abstract: 
 [-0.0745517784 + 0.0803511791*I, -0.0086691976 - 0.135608464*I, 0.1468419221 + 0.0266642165*I, -0.0212539276 - 0.0343654989*I, -0.0152000948 + 0.0507111149*I, 0.0473994576 + 0.0095710994*I, -0.0212539276 - 0.0343654989*I, -0.0152000948 + 0.0507111149*I, 0.0473994576 + 0.0095710994*I]
1.0000000000000102


In [78]:
# loss
manual_loss_list = [n*np.log(p) if p!= 0 else 0 for n,p in zip(n_fresh_norm, pr_an)]
manual_loss = -sum(manual_loss_list)
print('manual_loss_list: \n', manual_loss_list)
print('manual_loss: ', manual_loss)
print('loss at nice params: ', loss_nice)

manual_loss_list: 
 [-0.6108985742780509, -0.3318172636623165, -0.5725487803361318, -0.1180053397672549, -0.1734522409932759, -0.3467970756382234, -0.13174062962752361, -0.17627083990941664, -0.338096798944849]
manual_loss:  2.799627543157043
loss at nice params:  2.279530525594202


In [39]:
# opt_p = GD_new([0, m.pi/4, 0, 1, .9] , nit = 20, step = 1/10, nj_unnorm = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671] , N = 10000)

# new method to simply minimize the loss function directly without writing an explicit algorithm for it.

# first trying using the probabilities as variables instead of the parameters and seeing how accurate we can get and then later on trying with the parameters

# objective fn
# loss_new(vars, nj_unnorm = [842, 854,1615,799,800,1714,892,813,1671], lossfn ='old')


# define function for cross entropy of list n and list p

nju= [842, 854,1615,799,800,1714,892,813,1671]
njn = [i/sum(nju) for i in nju]
# njn = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]
def loss_t(p, n = [0.0842, 0.0854, 0.1615, 0.0799, 0.08, 0.1714, 0.0892, 0.0813, 0.1671]):
    lnl_p = [i * np.log(p) if p != 0 else 0 for i, p in zip(n, p)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.
    loss = sum([abs(i) for i in lnl_p])
    return loss
    


def constrain_p_sum(p):
    return (sum(p) - 1)


cons = {'type': 'eq', 'fun': constrain_p_sum}
ig = [1/9 for i in range(9)]
sol = GDlib(loss_t, ig, method='SLSQP', constraints=cons)

sol
# Result: . It worked amazing, the result was fantastic, it was matching up to like eighth decimal and the probabilities were also very very close. Although the fidelities could not be found just using these for that we need parameters that is the next part.

/var/folders/z5/8__h5b611fz3d61c954t7_540000gn/T/ipykernel_58481/4224283144.py:17: RuntimeWarning: invalid value encountered in log
  lnl_p = [i * np.log(p) if p != 0 else 0 for i, p in zip(n, p)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.1377889569311237
       x: [ 8.421e-02  8.542e-02  1.615e-01  7.991e-02  8.000e-02
            1.714e-01  8.920e-02  8.128e-02  1.671e-01]
     nit: 7
     jac: [-1.000e+00 -1.001e+00 -1.000e+00 -1.001e+00 -1.000e+00
           -9.998e-01 -9.995e-01 -9.988e-01 -1.000e+00]
    nfev: 77
    njev: 7

In [51]:
# Now, trying to find the parameters 
# first the inequality constraints for all of the five variables for them to remain in between 0 and pi by 1 for the first four variables and 0 and pi over 2 for the fifth variable

def c_1_g(params):
    return (m.pi + 1e-12 - params[0])               # 1e-12 is added to avoid the floating point error

def c_2_g(params):
    return (m.pi + 1e-12 - params[1])

def c_3_g(params):
    return (m.pi + 1e-12 - params[2])

def c_4_g(params):
    return (m.pi + 1e-12 - params[3])

def c_5_g(params):
    return (m.pi/2 + 1e-12 - params[4])

def c_1_l(params):
    return (params[0])

def c_2_l(params):
    return (params[1]+ 1e-12)

def c_3_l(params):
    return (params[2]+ 1e-12)

def c_4_l(params):  
    return (params[3]+ 1e-12)

def c_5_l(params):
    return (params[4]+ 1e-12)


cons_ineq = ({'type': 'ineq', 'fun': c_1_g}, {'type': 'ineq', 'fun': c_2_g}, {'type': 'ineq', 'fun': c_3_g}, {'type': 'ineq', 'fun': c_4_g}, {'type': 'ineq', 'fun': c_5_g}, {'type': 'ineq', 'fun': c_1_l}, {'type': 'ineq', 'fun': c_2_l}, {'type': 'ineq', 'fun': c_3_l}, {'type': 'ineq', 'fun': c_4_l}, {'type': 'ineq', 'fun': c_5_l})
ig = [m.pi/12, m.pi/3, 1, m.pi/4, m.pi/8]

sol = GDlib(loss_new, ig, method='SLSQP', constraints=cons_ineq, args = ([0, m.pi/2, 0, 0, m.pi/4], 10000, xxnn, 'new'))

# sol = GDlib(loss_new, ig, method='SLSQP', args = ([0, m.pi/2, 0, 0, m.pi/4], 10000, nju, 'old'))
print(sol)
print('fid: ', fn.fid(sol.x, [0, m.pi/2, 0, 0, m.pi/4]))





 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.36169035943175454
       x: [ 1.314e-01  1.358e+00  1.495e+00  1.254e+00  7.609e-01]
     nit: 57
     jac: [-2.096e-02 -1.718e-01  2.293e-03 -8.954e-03  3.853e-01]
    nfev: 409
    njev: 57
fid:  [0.9828217580281529, 0.9555014482984382]


In [43]:
p_inv = [45.413971053989975, 44.97208496779245, 0.5324424926602901, -179.92245596910786, (44.65140134482599-7.67504195354202e-17j)]
p_inv_rad = [i*(m.pi/180) for i in p_inv]

# Now, trying to find the corresponding cross entropy or the loss function for the inversion parameters

xxn = fn.num_experiment(params= p_inv_rad, N = 10000, an_pr= True)[4]
xxnn = [i/sum(xxn) for i in xxn]
print(xxn)
inv_loss = loss_new(p_inv_rad, nj_unnorm = xxn , lossfn ='old')

print('min', loss_t(xxnn, xxnn))
inv_loss



[1188, 1304, 0, 1258, 1250, 1216, 1248, 1299, 1237]
min 2.079019206430704


2.0794782168015233

In [45]:
xxn = [1188, 1304, 0, 1258, 1250, 1216, 1248, 1299, 1237]
xxnn = [i/sum(xxn) for i in xxn]

# Now, trying to optimize using the probability parameters method to compare the optimization results with the inversion method

ig = [1/9 for i in range(9)]
sol = GDlib(loss_t, ig, method='SLSQP', constraints=cons, args= (xxnn))

sol



/var/folders/z5/8__h5b611fz3d61c954t7_540000gp/T/ipykernel_36110/4224283144.py:17: RuntimeWarning: invalid value encountered in log
  lnl_p = [i * np.log(p) if p != 0 else 0 for i, p in zip(n, p)]                          # manually putting zero for cases of p_i*log(n_i) , if any n_i are 0.


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.079029317672689
       x: [ 1.195e-01  1.294e-01  4.289e-07  1.260e-01  1.253e-01
            1.217e-01  1.251e-01  1.291e-01  1.239e-01]
     nit: 21
     jac: [-9.941e-01 -1.008e+00  0.000e+00 -9.980e-01 -9.978e-01
           -9.991e-01 -9.978e-01 -1.006e+00 -9.982e-01]
    nfev: 297
    njev: 21

In [41]:
martin_opt_case1 = [1.5486207416093, 3.1274442725918, 4.7643344242595, 0.21847560771103, 0.77864789419447]
martin_opt_case2 = [2.6072033366248, 4.1823366657766, 3.9196739422848, 1.5574885302675, 0.54017337241981]
tru_2 = [0.5235987755982988, 2.0943951023931953, 0.7853981633974483, 1.5707963267948966, 0.5235987755982988] 
njn1 = [842, 854,1615,799,800,1714,892,813,1671]
njn2 = [654, 558, 1169, 967,  2313, 521,  1260, 1152,1406]

nju1 = [i/sum(njn1) for i in njn1]
nju2 = [i/sum(njn2) for i in njn2]

loss1martin = loss_new(martin_opt_case1, nj_unnorm = njn1, lossfn = 'old')
loss2martin = loss_new(martin_opt_case2, nj_unnorm = njn2, lossfn = 'old')

print(loss1martin, loss2martin)
print([i for i in martin_opt_case1[:4]])

NameError: name 'loss_new' is not defined

In [20]:
# degrees
print([i*(180/m.pi) for i in martin_opt_case1])
print([i*(180/m.pi) for i in martin_opt_case2])
print([i*(180/m.pi) for i in tru_2])

# print(m.pi/4)
fn.fid(martin_opt_case1, [m.pi/2, 0, 0, 0, m.pi/4])

[88.72943256063249, 179.1893574818719, 272.97625469896036, 12.517730248397843, 44.61323806409221]
[149.3817475210271, 239.63023945181598, 224.58077396032405, 89.23751942436132, 30.94965444500354]
[29.999999999999996, 119.99999999999999, 45.0, 90.0, 29.999999999999996]


[0.9995083240245686, 0.9997998366716809]

In [11]:
loss_t(nju2, nju2)

2.0945855608529884

In [20]:
pr_martin2 = [i(*martin_opt_case2) for i in pr_num]

print(nju2)
print(pr_martin2)


# loss_t(pr_martin2, nju2)

[0.0654, 0.0558, 0.1169, 0.0967, 0.2313, 0.0521, 0.126, 0.1152, 0.1406]
[0.06793312233662688, 0.05364199613778085, 0.11887112345186476, 0.16988019120071673, 0.2546207101352122, 0.053578937935631316, 0.051837943260331074, 0.09371230486351896, 0.1359236706783202]


In [40]:
check_martin = [0.5343893173506,1.0407440166693,0.7780813019974701,4.6990812421347,2.6014192818704]

#print above but in a single line each
print('\n cos theta_j',m.cos(check_martin[0]), m.cos(check_martin[1]))
print('sin theta_j',m.sin(check_martin[0]), m.sin(check_martin[1]))
print(' cos^2, sin^2 alpha',m.cos(check_martin[4])**2, m.sin(check_martin[4])**2)

# probs

pr_martin2_check = [i(*check_martin) for i in pr_num]
pr_martin2_check

Loss_check = loss_t(pr_martin2_check, nju2)
Loss_check


0.860579819920731 
 0.5055784740593927
0.5093155932672808 
 0.8627806248217307
0.7355112613814505 
 0.26448873861854955

 cos theta_j 0.860579819920731 0.5055784740593927
sin theta_j 0.5093155932672808 0.8627806248217307
 cos^2, sin^2 alpha 0.7355112613814505 0.26448873861854955


2.094972152688196

In [21]:
#fidelity function using parameters
def fid(params1 = [0,m.pi/2,0,0,m.pi/4], params2 = [0,m.pi/2,0,0,m.pi/4], coeff_mode = False ):
    #if simply the coeffs 4-lists are given
    if coeff_mode == True:
        fid0 = (abs(sum([np.conj(params1[i])*params2[i] for i in range(0,2)])))**2
        fid1 = (abs(sum([np.conj(params1[i])*params2[i] for i in range(2,4)])))**2
    else:               # generating the qm states from the parameters to find the fidelities
        states1 = fn.Creating_states(params1)[0]
        states2 = fn.Creating_states(params2)[0]
        # Calculate the fidelities between the corresponding states
        fid0_sym = (abs(states1[0].H * states2[0])**2).evalf()
        fid1_sym = (abs(states1[1].H * states2[1])**2).evalf()
        # Extract the real part of the fidelity values
        fid0 = complex(fid0_sym[0]).real
        fid1 = complex(fid1_sym[0]).real
    return [fid0, fid1]

In [35]:
tru_deg = [29.999999999999996, 59.99999999999999, 29.999999999999996, 0.0, 29.999999999999996]
params_inv_deg =  [29.638608229696608, 62.537229284179226, 25.117537308310606, -16.030328746515185, (28.435771189456087+1.6630221342149307e-15j)]
params_opt = [(62.052359057321794+0j), (30.03131138748132+0j), (-0.2614468121347648+0j), (29.15852708313148+0j), (59.32744200079733+2.9025211775593585e-17j)]

# in radians
tru = [i*(m.pi/180) for i in tru_deg]
params_inv = [i*(m.pi/180) for i in params_inv_deg]
params_opt = [i*(m.pi/180) for i in params_opt]

# fid([0, m.pi/2, 0, 0, m.pi/4], [0, m.pi/2, 0, 0, m.pi/4])
print(fid(tru, params_inv))
print(fid([m.pi/3 ,m.pi/6, 0, m.pi/6, m.pi/6], params_opt))

[0.9986095076327963, 0.9842604563694797]
[0.9987137132359978, 0.9999592342419349]


In [42]:

print(fn.fid(check_martin, tru_2))
#ppm error
ppm = [(1-i)*1e6 for i in fn.fid(check_martin, tru_2)]
ppm


[0.999873407960869, 0.9999249028120026]


[126.59203913101268, 75.09718799736387]

In [7]:


import numpy as np

# Original matrix
A = np.array([[1, 2, 3],
              [0, 4, 5],
              [0, 0, 0]])

# Compute eigenvalues and eigenvectors
eigvals, eigvecs = np.linalg.eig(A)

print("Original Eigenvalues:\n", eigvals)
print("Original Eigenvectors:\n", eigvecs)

# Perturbed matrix
perturbation = 0.02
A_perturbed = A - perturbation * np.array([[1,1,1],[0,1,1],[1,0,1]])

print(A_perturbed)

# Compute eigenvalues and eigenvectors of the perturbed matrix
eigvals_perturbed, eigvecs_perturbed = np.linalg.eig(A_perturbed)

print("\nPerturbed Eigenvalues:\n", eigvals_perturbed)
print("Perturbed Eigenvectors:\n", eigvecs_perturbed)

Original Eigenvalues:
 [1. 4. 0.]
Original Eigenvectors:
 [[ 1.          0.5547002  -0.2981424 ]
 [ 0.          0.83205029 -0.74535599]
 [ 0.          0.          0.59628479]]
[[ 0.98  1.98  2.98]
 [ 0.    3.98  4.98]
 [-0.02  0.   -0.02]]

Perturbed Eigenvalues:
 [ 0.98623402 -0.00972338  3.96348936]
Perturbed Eigenvectors:
 [[ 0.99925672 -0.3058711   0.55103826]
 [ 0.03303844 -0.74302732  0.83447539]
 [-0.01986132  0.5952758  -0.00276661]]


In [8]:
import sympy as sp

# Define the symbolic matrix
rho = sp.Matrix([[0.25 + 0.00000000e+00j, -0.0625-1.87500000e-01j, -0.1875+9.91116524e-02j],
                 [-0.0625+1.87500000e-01j, 0.25 - 1.58794886e-34j, -0.1875-9.91116524e-02j],
                 [-0.1875-9.91116524e-02j, -0.1875+9.91116524e-02j, 0.5   +5.02387877e-35j]])

# Compute eigenvalues and eigenvectors
eigenvalues_sympy, eigenvectors_sympy = rho.eigenvals(), rho.eigenvects()

# Print the eigenvalues
print("Eigenvalues (SymPy):", eigenvalues_sympy)

# Extract and print the eigenvectors
for eigenvalue, multiplicity, vectors in eigenvectors_sympy:
    for vector in vectors:
        print(f"Eigenvector for eigenvalue {eigenvalue}: {vector}")

Eigenvalues (SymPy): {1.74575537758787e-11 - 4.61562975474726e-33*I: 1, 0.242082882894236 + 1.86389228904848e-34*I: 1, 0.757917117088306 + 4.88059210183399e-33*I: 1}
Eigenvector for eigenvalue 1.74575537758787e-11 - 4.61562975474726e-33*I: Matrix([[-0.50855948606321 + 0.421304472669645*I], [-0.0616986117254196 + 0.657513110795123*I], [-0.167024729805028 + 0.315978354513266*I]])
Eigenvector for eigenvalue 0.242082882894236 + 1.86389228904848e-34*I: Matrix([[-0.477883330600306 - 0.339811179657743*I], [0.549940977995031 + 0.203492103551704*I], [0.261163337259709 - 0.494070319184744*I]])
Eigenvector for eigenvalue 0.757917117088306 + 4.88059210183399e-33*I: Matrix([[-0.464897140471242 + 0.0624205394025948*I], [-0.111854368854807 - 0.455537347673867*I], [0.618324878398021 + 0.42145343139993*I]])
